In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.shape[0]

2584242

In [5]:
df_copy = df.copy()

In [7]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1980_2021 = [df_copy['yearOfLoss'].min(), 1990, 2000, 2010, 2020, df_copy['yearOfLoss'].max() + 1]
labels_1980_2021 = [1980, 1990, 2000, 2010, 2020]

df_copy['yearOfLoss_1990_2021'] = pd.cut(df_copy['yearOfLoss'], bins=bins_1980_2021, labels=labels_1980_2021, right=False).astype(int)

In [8]:
df_copy['yearOfLoss_1990_2021'].value_counts()

2000    705871
2010    696568
1990    529694
1980    510616
2020    141493
Name: yearOfLoss_1990_2021, dtype: int64

# Checking for missing shapefiles after dropping NA of the mentioned geographic unit..

##### The units are: ['state',  'latitude', 'longitude', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips']

### 1. State

#### We are assuming that the geometries of state didn't change for our dataset (According to Wikipedia article (https://en.wikipedia.org/wiki/Territorial_evolution_of_the_United_States) all the changes to US map are either outside the mainland or too small to map. No missing shapefile for states.

### 2. Latitude/Longitude

In [ ]:
test = df_copy[df_copy['latitude'].notna() & df_copy['longitude'].notna()]

In [ ]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique combinations of lat/long:', test.drop_duplicates(subset=['latitude', 'longitude']).shape[0])

In [ ]:
print('Value_counts of combinations of lat/long per decade:\n', test.drop_duplicates(subset=['latitude', 'longitude'])['yearOfLoss_1990_2021'].value_counts())

#### Since latitude and longitude doesn't change over time, we are able to generate all the respective geometries`

### 3. reportedZipCode

In [22]:
test = df[df['reportedZipCode'].notna()]

In [23]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique zipcodes:', test.drop_duplicates(subset=['reportedZipCode']).shape[0])

Total observations available: 2528415
Observations lost: 55827
Percentage NA: 2.1602852983582808
Total unique zipcodes: 25952


##### Now we can calculate for each decade how many of the zipcodes have shapefiles with geometry attribute. For zipcodes we only have shapefiles available post 2000 (inclusive) in open source.

In [24]:
# Read shapefile of zipcode

chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 100000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

zipcode_df['year'] = zipcode_df['year'].replace({2012: 2010, 2021: 2020})
zipcode_df = zipcode_df.drop_duplicates(subset=['ZIPcode', 'year'])

In [25]:
df_geographic_unique = df[['reportedZipCode', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['reportedZipCode'])

In [26]:
df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].dropna().astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

In [27]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [28]:
# Filter for post 2000 since we don't have shapefiles pre 2000 for zipcodes
df_geographic_unique = df_geographic_unique[df_geographic_unique['yearOfLoss_1990_2021']!=1990]

In [29]:
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)

In [30]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'yearOfLoss_1990_2021']).index.map(zipcode_df['geometry_zipcode'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_zipcode'].isna()

zipcode_df.reset_index(inplace=True)

In [31]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test.drop_duplicates(subset=['reportedZipCode', 'yearOfLoss_1990_2021']).shape[0])
print('Number of reportedZipCode per decade without a matching geometry')
print(test.drop_duplicates(subset=['reportedZipCode', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Total missing: 1819
Number of reportedZipCode per decade without a matching geometry
2000    1223
2010     517
2020      79
Name: yearOfLoss_1990_2021, dtype: int64


In [32]:
df_test = df_copy[df_copy['reportedZipCode'].notna()]

df_test['reportedZipCode'] = df_test['reportedZipCode'].dropna().astype(int).astype(str)
df_test['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_test['reportedZipCode']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_25720\4112593955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['reportedZipCode'] = df_test['reportedZipCode'].dropna().astype(int).astype(str)
C:\Users\Asus\AppData\Local\Temp\ipykernel_25720\4112593955.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_test['reportedZipCode']]


In [33]:
print('Value_counts of reportedZipCode per decade:\n', df_test.drop_duplicates(subset=['reportedZipCode', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Value_counts of reportedZipCode per decade:
 1990    20965
2010    17449
2000    16263
2020     8757
Name: yearOfLoss_1990_2021, dtype: int64


In [34]:
test.set_index(['reportedZipCode', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['reportedZipCode', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same reportedZipCode and yearOfLoss_1990_2021 as in test.")

There are 13098 rows in df_copy with the same reportedZipCode and yearOfLoss_1990_2021 as in test.


In [35]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

2000    10810
2010     2140
2020      148
Name: yearOfLoss_1990_2021, dtype: int64

In [36]:
df_copy[df_copy['yearOfLoss_1990_2021']==1990].shape[0]

1040310

### 4. countyCode

In [ ]:
test = df[df['countyCode'].notna()]

In [ ]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique countyCode:', test.drop_duplicates(subset=['countyCode']).shape[0])

##### Now we can calculate for each decade how many of the county code have shapefiles with geometry attribute.

In [ ]:
# Read shapefile of county code
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

County_df['year'] = County_df['year'].replace({2012: 2010, 2021: 2020})
County_df = County_df.drop_duplicates(subset=['CountyID', 'year'])

In [ ]:
df_geographic_unique = df[['countyCode', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['countyCode'])

In [ ]:
df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censuscounty.zfill(5) for censuscounty in df_geographic_unique['countyCode']]

In [ ]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [ ]:
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)

In [ ]:
County_df.set_index(['CountyID', 'year'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode', 'yearOfLoss_1990_2021']).index.map(County_df['geometry_county'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_county'].isna()

County_df.reset_index(inplace=True)

In [ ]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test.drop_duplicates(subset=['countyCode', 'yearOfLoss_1990_2021']).shape[0])
print('Number of County per decade without a matching geometry')
print(test.drop_duplicates(subset=['countyCode', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

In [ ]:
df_test = df_copy[df_copy['countyCode'].notna()]

df_test['countyCode'] = [str(int(float(i))) for i in df_test['countyCode']]
df_test['countyCode'] = [censuscounty.zfill(5) for censuscounty in df_test['countyCode']]

In [ ]:
print('Value_counts of county per decade:\n', df_test.drop_duplicates(subset=['countyCode', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

In [ ]:
test.set_index(['countyCode', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['countyCode', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same countyCode and yearOfLoss_1990_2021 as in test.")

In [ ]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

### 5. censusTract

In [ ]:
test = df[df['censusTract'].notna()]

In [ ]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique censusTract:', test.drop_duplicates(subset=['censusTract']).shape[0])

##### Now we can calculate for each decade how many of the census tract have shapefiles with geometry attribute.

In [ ]:
# Read shapefile of county code
chunk_size = 30000 
chunks = [x for x in range(0, 180000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

In [ ]:
df_geographic_unique = df[['censusTract', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['censusTract'])

In [ ]:
df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [ ]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [ ]:
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

In [ ]:
Tract_df.set_index(['GEOID', 'year'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract', 'yearOfLoss_1990_2021']).index.map(Tract_df['geometry_tract'])

# Find rows with NaN values
na_rows = df_geographic_unique['geometry_tract'].isna()

Tract_df.reset_index(inplace=True)

In [ ]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test.drop_duplicates(subset=['censusTract', 'yearOfLoss_1990_2021']).shape[0])
print('Number of Tract per decade without a matching geometry')
print(test.drop_duplicates(subset=['censusTract', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

In [ ]:
df_test = df_copy[df_copy['censusTract'].notna()]

df_test['censusTract'] = [str(int(float(i))) for i in df_test['censusTract']]
df_test['censusTract'] = [censusBG.zfill(11) for censusBG in df_test['censusTract']]

In [ ]:
print('Value_counts of censusTract per decade:\n', df_test.drop_duplicates(subset=['censusTract', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

In [ ]:
test.set_index(['censusTract', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['censusTract', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same censusTract and yearOfLoss_1990_2021 as in test.")

In [ ]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

### 6. censusBlockGroupFips

In [20]:
test = df[df['censusBlockGroupFips'].notna()]

In [21]:
print('Total observations available:', test.shape[0])
print('Observations lost:', df.shape[0]-test.shape[0])
print('Percentage NA:', (df.shape[0]-test.shape[0])*100/df.shape[0])
print('Total unique censusBlockGroup:', test.drop_duplicates(subset=['censusBlockGroupFips']).shape[0])

Total observations available: 2447264
Observations lost: 136978
Percentage NA: 5.300509781978623
Total unique censusBlockGroup: 106786


##### Now we can calculate for each decade how many of the census tract have shapefiles with geometry attribute.

In [8]:
chunk_size = 40000 
chunks = [x for x in range(0, 320000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [9]:
df_geographic_unique = df[['censusBlockGroupFips', 'yearOfLoss']].drop_duplicates()
df_geographic_unique = df_geographic_unique.dropna(subset=['censusBlockGroupFips'])

In [10]:
df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

In [11]:
# Define bins and labels for yearOfLoss_1990_2021
bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique = df_geographic_unique.drop(columns='yearOfLoss')

In [12]:
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)

In [13]:
BG_df.set_index(['GEOID', 'year'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021']).index.map(BG_df['geometry_BG'])

# Find rows with NaN values 
na_rows = df_geographic_unique['geometry_BG'].isna()

BG_df.reset_index(inplace=True)

In [14]:
test = df_geographic_unique.loc[na_rows, ]
print('Total missing:', test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021']).shape[0])
print('Number of censusBlockGroupFips per decade without a matching geometry')
print(test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Total missing: 55299
Number of censusBlockGroupFips per decade without a matching geometry
1990    31615
2000    17045
2020     6373
2010      266
Name: yearOfLoss_1990_2021, dtype: int64


In [15]:
df_test = df_copy[df_copy['censusBlockGroupFips'].notna()]

df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_25720\889359098.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [str(int(float(i))) for i in df_test['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_25720\889359098.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_test['censusBlockGroupFips']]


In [16]:
print('Value_counts of censusBlockGroupFips per decade:\n', df_test.drop_duplicates(subset=['censusBlockGroupFips', 'yearOfLoss_1990_2021'])['yearOfLoss_1990_2021'].value_counts())

Value_counts of censusBlockGroupFips per decade:
 1990    71925
2010    61534
2000    52227
2020    24273
Name: yearOfLoss_1990_2021, dtype: int64


In [17]:
test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)
df_test.set_index(['censusBlockGroupFips', 'yearOfLoss_1990_2021'], inplace=True)

# Select rows of df_copy that have indices found in test
matching_rows = df_test.loc[df_test.index.isin(test.index)]

matching_rows.reset_index(inplace=True)
test.reset_index(inplace=True)
df_test.reset_index(inplace=True)

print(f"There are {matching_rows.shape[0]} rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.")

There are 682098 rows in df_copy with the same censusBlockGroupFips and yearOfLoss_1990_2021 as in test.


In [18]:
matching_rows['yearOfLoss_1990_2021'].value_counts()

1990    438104
2000    198715
2020     44855
2010       424
Name: yearOfLoss_1990_2021, dtype: int64